In [1]:
import pandas as pd
import numpy as np

from sklearn.cluster import AgglomerativeClustering
import sqlalchemy as sal
from sqlalchemy import text

%matplotlib inline

In [ ]:
# csv paths
nutrient_matrix_csv_p = r"../../data/Nutrients_Branded_Foods_2018/nutrients_matrix.csv.gz"

# nourish db connection config
nourish_user = ""

nourish_pswd = ""

#### Import cleaned nutrient data. Set the NDB Number to the index. Remove zero rows.

In [10]:
#read data
nutrients_matrix = pd.read_csv(nutrient_matrix_csv_p)
#set ndb no as index
nutrients_matrix.set_index("NDB_No", inplace = True)
#create a list of indices where all entries are zero
nutrients_zero_rows = nutrients_matrix.loc[(nutrients_matrix==0).all(axis=1)].index
#filter out indicies where all entires are zero
nutrients_matrix = nutrients_matrix[~nutrients_matrix.index.isin(nutrients_zero_rows)]
print(nutrients_matrix.shape)
nutrients_matrix.head()

(234247, 94)


,"Calcium, Ca","Carbohydrate, by difference",Cholesterol,Energy,"Fatty acids, total saturated","Fatty acids, total trans","Fiber, total dietary","Iron, Fe",Protein,"Sodium, Na",...,18:2 undifferentiated,Glucose (dextrose),Starch,Lutein + zeaxanthin,"Fluoride, F","18:2 n-6 c,c","Choline, from phosphotidyl choline","18:3 n-3 c,c,c (ALA)",Epigallocatechin-3-gallate,Vitamin D3 (cholecalciferol)
NDB_No,,,,,,,,,,,,,,,,,,,,,
45001524,50.0,35.00,25.0,200.0,3.75,0.0,0.0,0.0,2.5,75.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
45001528,0.0,43.24,0.0,162.0,0.00,0.0,0.0,0.0,0.0,703.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
45001529,0.0,41.18,0.0,176.0,0.00,0.0,0.0,0.0,0.0,676.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
45001530,0.0,34.29,0.0,143.0,0.00,0.0,0.0,0.0,0.0,971.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
45001531,0.0,45.95,0.0,189.0,0.00,0.0,0.0,0.0,0.0,757.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


#### Import the product data for reference

In [3]:
engine = sal.create_engine('postgresql+psycopg2://' + nourish_user + ':' + nourish_pswd + '@awesome-hw.sdsc.edu/nourish')
conn = engine.connect()

In [4]:
query_products = text('''SELECT "NDB_Number", "long_name", "manufacturer" From "Products_Branded_Foods_2018" ''')
result = conn.execute(query_products)
product_data = [i for i in result]
product_data = pd.DataFrame(product_data)
product_data.set_index("NDB_Number", inplace = True)
product_data.head(2)

,long_name,manufacturer
NDB_Number,,
45265623,MINI CHURROS,"Nong Shim Co., Ltd."
45265624,CURLY CANES,"Frankford Candy, LLC"


Example of product with zero values

In [11]:
product_data[product_data.index.isin(nutrients_zero_rows)]

,long_name,manufacturer
NDB_Number,,
45265922,NATURAL ENERGY PREMIUM BLACK TEA,Lipton
45266011,APPLE CIDER VINEGAR,The Moran Group Incorporated
45266216,SELTZER,Polar Corporation
45266289,APPLE CIDER VINEGAR,Pastorelli Food Products Inc.
45266419,NATURALLY FLAVOR ED SPARKLING WATER BEVERAGE,"Hy-Vee, Inc."
...,...,...
45264628,HARISSA,Morton Bassett Inc.
45264761,ORGANIC APPLE CIDER VINEGAR,HEINEN'S
45264840,COLD BREW COFFEE,Rainbow Games Inc


#### Do Hierarchical clustering

Create a sample of the main dataset for performance

In [23]:
sample = nutrients_matrix.sample(frac = 0.10, random_state=1)
print(sample.shape)
sample.head(1)

(23425, 94)


,"Calcium, Ca","Carbohydrate, by difference",Cholesterol,Energy,"Fatty acids, total saturated","Fatty acids, total trans","Fiber, total dietary","Iron, Fe",Protein,"Sodium, Na",...,18:2 undifferentiated,Glucose (dextrose),Starch,Lutein + zeaxanthin,"Fluoride, F","18:2 n-6 c,c","Choline, from phosphotidyl choline","18:3 n-3 c,c,c (ALA)",Epigallocatechin-3-gallate,Vitamin D3 (cholecalciferol)
NDB_No,,,,,,,,,,,,,,,,,,,,,
45197846,0.0,22.78,38.0,228.0,1.27,0.0,1.3,1.37,11.39,494.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


Since we have sparse data, use cosine to take advantage of the dot product which would penalize any zero entries by lowering the similarity score. Use average linkage as ward is not available for this case.

In [24]:
%%time
agglo_cluster = AgglomerativeClustering(n_clusters = None, metric = "cosine", compute_full_tree = 'auto',
                                        linkage = 'average', distance_threshold = 0.3).fit(sample)

CPU times: total: 28.2 s
Wall time: 28.2 s


In [29]:
#merge the cluster labels with the sample dataframe
sample['labels'] = agglo_cluster.labels_
products_labeled = product_data.merge(sample['labels'], left_index = True, right_index = True).copy(deep = True)
products_labeled.sort_values(by = ['labels', 'long_name'], inplace = True)

In [ ]:
# products_labeled.to_csv(r"../../data/Nutrients_Branded_Foods_2018/hrchy_clstr_sample0.10_cosine0.30_avg_lnkge.csv", 
#                    index = True)

In [26]:
np.unique(agglo_cluster.labels_)

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33,
       34, 35, 36, 37, 38], dtype=int64)

In [27]:
products_labeled['labels'].value_counts()

8     10145
0      5159
15     4925
1      1534
22      760
5       439
4       102
12       71
13       62
10       46
14       29
2        29
6        22
28       19
24       12
11       11
34        6
3         6
16        5
26        5
19        4
17        4
32        3
9         3
7         3
20        3
21        2
18        2
36        2
33        2
37        1
35        1
30        1
31        1
29        1
27        1
25        1
23        1
38        1
Name: labels, dtype: int64

In [42]:
products_labeled[products_labeled['labels'] == 14]

,long_name,manufacturer,labels
45237613,COCONUT PINEAPPLE NATURALLY FLAVORED SPARKLING...,"Talking Rain Beverage Co., Inc",14
45137568,"CUMBERLAND FARMS, ZERO CALORIES SPARKLING BEVE...",Cumberland Farms Dairy Inc,14
45237325,FLAVORED WATER BEVERAGE,"CVS Pharmacy, Inc.",14
45145188,FRUIT REFRESH SPARKLING WATER BEVERAGE,Krause Gentle Corporation,14
45163716,"HANNAFORD, CHILL SPARKLING WATER BEVERAGE, BLA...",Hannaford Bros. Co.,14
45152266,"HARRIS TEETER, SPARKLING WATER BEVERAGE, COCONUT",Harris-Teeter Inc.,14
45252018,"HILAND PREMIUM, FRUIT REFRESH, SPARKLING WATER...",Krause Gentle Corporation,14
45240646,ICED TEA BEVERAGE,"Talking Rain Beverage Co., Inc",14
45237609,"KEVIN DURANT KD 35 LIMITED EDITION, NATURALLY ...",SPARKLING ICE,14
45161302,"KIST, SPARKLING WATER WITH NATURAL ENERGY, ACA...","Intrastate Distributors, Inc.",14


In [36]:
sample.shape

(23425, 95)

In [38]:
agglo_cluster.children_

array([[10909, 22790],
       [ 1074, 16773],
       [ 8988, 18027],
       ...,
       [33008, 46845],
       [ 7198, 46846],
       [ 4899, 46847]])